# Setting up RGB LED -> PMODB PINs

In [1]:
from pynq.overlays.base import BaseOverlay
import time
from datetime import datetime
base = BaseOverlay("base.bit")

In [9]:
%%microblaze base.PMODB

#include "gpio.h"
#include "pyprintf.h"

//Function to turn on/off a selected pin of PMODB
void write_gpio(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
}

In [10]:
#blink each color for 1 sec

write_gpio(1, 1) #blue
time.sleep(1)
write_gpio(1, 0)
time.sleep(1)
write_gpio(2, 1) #green
time.sleep(1)
write_gpio(2, 0)
time.sleep(1)
write_gpio(3, 1) #red
time.sleep(1)
write_gpio(3, 0)
time.sleep(1)

# Initialize/Emulate/Test PWM for optimal frequency

In [11]:
base = BaseOverlay("base.bit")

In [2]:
%%microblaze base.PMODB

#include "gpio.h"
#include "pyprintf.h"

//convert frequency to time period with 25% dutcy cycle
float set_hi_pulse(float freq, float dc){
   return (1/freq)*dc;
}
float set_low_pulse(float freq, float dc){
   return (1/freq)*(100-dc);
}


//Function to turn on/off a selected pin of PMODA
void write_gpio(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
}

//all LED's off
void all_off(){
    for (int i=1; i<4; i++){
        gpio pin_out = gpio_open(i);
        gpio_set_direction(pin_out, GPIO_OUT);
        gpio_write(pin_out, 0); //off
    }
}

In [3]:
init_freq = 2500 #Hz
dc = 25 #%

for a in range(6):
    freq = init_freq+(a*100)
    high = set_hi_pulse(freq,dc)
    low = set_low_pulse(freq,dc)
    print(f"freq: {freq}\thigh pulse: {high}\tlow pulse: {low}")
    
    #begin with all LED's off
    all_off()
    
    for b in range(50):
        write_gpio(1,1)
        time.sleep(high)
        write_gpio(1,0)
        time.sleep(low)
    
    time.sleep(0.5)

freq: 2500	high pulse: 0.009999999776482582	low pulse: 0.029999999329447746
freq: 2600	high pulse: 0.009615384973585606	low pulse: 0.028846153989434242
freq: 2700	high pulse: 0.009259259328246117	low pulse: 0.02777777798473835
freq: 2800	high pulse: 0.0089285708963871	low pulse: 0.02678571455180645
freq: 2900	high pulse: 0.008620689623057842	low pulse: 0.0258620698004961
freq: 3000	high pulse: 0.00833333283662796	low pulse: 0.02500000037252903


# Initialize/Emulate/Test PWM for varying LED brightness
Vary Duty cycle for Visually perceived 25/50/75/100%

In [1]:
from pynq.overlays.base import BaseOverlay
import asyncio

base = BaseOverlay("base.bit")
btns = base.btns_gpio
#sw=base.switches_gpio

In [2]:
%%microblaze base.PMODB

#include "gpio.h"
#include "pyprintf.h"

//convert frequency to time period with varying dutcy cycle
float set_hi_pulse(float freq, float dc){
   return (1/freq)*dc;
}
float set_low_pulse(float freq, float dc){
   return (1/freq)*(100-dc);
}


//Function to turn on/off a selected pin of PMODA
int write_gpio(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
    return 0;
}

//all LED's off
int all_off(){
    for (int i=1; i<4; i++){
        gpio pin_out = gpio_open(i);
        gpio_set_direction(pin_out, GPIO_OUT);
        gpio_write(pin_out, 0); //off
    }
    return 0;
}

In [6]:
freq = 2900 #Hz

cond=0

async def flash_leds():
    global cond
    dc=0
    high = set_hi_pulse(freq,dc)
    low = set_low_pulse(freq,dc)
    while True:
        if cond==0:
            #Visually Perceived 25% LED brightness
            dc=34
            high = set_hi_pulse(freq,dc)
            low = set_low_pulse(freq,dc)
        elif cond==1:
            #Visually Perceived 50% LED brightness
            dc=58
            high = set_hi_pulse(freq,dc)
            low = set_low_pulse(freq,dc)
        elif cond==2:
            #Visually Perceived 75% LED brightness
            dc=78
            high = set_hi_pulse(freq,dc)
            low = set_low_pulse(freq,dc)
        elif cond==3:
            #Visually Perceived 100% LED brightness
            dc=100
            high = set_hi_pulse(freq,dc)
            low = set_low_pulse(freq,dc)
        write_gpio(1,1)
        await asyncio.sleep(high)
        write_gpio(1,0)
        await asyncio.sleep(low)

async def get_btns(_loop):
    global cond
    while True:
        await asyncio.sleep(0.01)
        if btns[0].read() != 0: #25% perceived LED brightness
            if btns[1].read() != 0: #OFF -- press both Button 0 and 1
                #all LED's off
                all_off()
                _loop.stop()
            else:
                cond = 0
        await asyncio.sleep(0.01)
        if btns[1].read() != 0: #50% perceived LED brightness
            cond = 1
        await asyncio.sleep(0.01)
        if btns[2].read() != 0: #75% perceived LED brightness
            cond = 2
        await asyncio.sleep(0.01)
        if btns[3].read() != 0: #100% perceived LED brightness
            cond = 3
        await asyncio.sleep(0.01)

loop = asyncio.new_event_loop()
loop.create_task(flash_leds())
loop.create_task(get_btns(loop))
loop.run_forever()
loop.close()
print("Done.")

Done.


# Varying duty cycles and LED brightness and human eye perception
Weber-Fechner Law

In [9]:
from pynq.overlays.base import BaseOverlay
import time
base = BaseOverlay("base.bit")

In [11]:
#use the microblaze from "Initialize/Emulate/Test PWM for varying LED brightness"

freq = 2900 #Hz
init_dc = 0 #%

for a in range(51):
    dc = init_dc+(a*2)
    high = set_hi_pulse(freq,dc)
    low = set_low_pulse(freq,dc)
    print(f"duty cycle: {dc}\thigh pulse: {high}\tlow pulse: {low}")
    
    #begin with all LED's off
    all_off()
    
    for b in range(30):
        write_gpio(1,1)
        time.sleep(high)
        write_gpio(1,0)
        time.sleep(low)
    
    time.sleep(0.5)

duty cycle: 0	high pulse: 0.0	low pulse: 0.03448275849223137
duty cycle: 2	high pulse: 0.0006896551931276917	low pulse: 0.03379310294985771
duty cycle: 4	high pulse: 0.0013793103862553835	low pulse: 0.03310345113277435
duty cycle: 6	high pulse: 0.002068965695798397	low pulse: 0.032413795590400696
duty cycle: 8	high pulse: 0.002758620772510767	low pulse: 0.03172414004802704
duty cycle: 10	high pulse: 0.003448275849223137	low pulse: 0.03103448450565338
duty cycle: 12	high pulse: 0.004137931391596794	low pulse: 0.030344828963279724
duty cycle: 14	high pulse: 0.004827586468309164	low pulse: 0.029655173420906067
duty cycle: 16	high pulse: 0.005517241545021534	low pulse: 0.02896551787853241
duty cycle: 18	high pulse: 0.006206896621733904	low pulse: 0.028275862336158752
duty cycle: 20	high pulse: 0.006896551698446274	low pulse: 0.027586206793785095
duty cycle: 22	high pulse: 0.007586207240819931	low pulse: 0.026896553114056587
duty cycle: 24	high pulse: 0.008275862783193588	low pulse: 0.02620

# Button changes LED color
Button 0: Red
Button 1: Green
Button 2: Blue
Button 3: turn LEDs OFF

In [13]:
from pynq.overlays.base import BaseOverlay
import asyncio
import time

In [14]:
base = BaseOverlay("base.bit")
btns = base.btns_gpio

In [16]:
#use the microblaze from "Initialize/Emulate/Test PWM for varying LED brightness"

RED = 3
GREEN = 2
BLUE = 1
rgb = 0

freq = 2900 #Hz
dc=25 #%
high = set_hi_pulse(freq,dc)
low = set_low_pulse(freq,dc)

cond=True

async def flash_red():
    all_off()
    global cond
    while cond:
        write_gpio(RED,1)
        await asyncio.sleep(1)
        write_gpio(RED,0)
        await asyncio.sleep(1)
    
async def flash_leds():
    global rgb
    while True:
        write_gpio(rgb,1)
        await asyncio.sleep(high)
        write_gpio(rgb,0)
        await asyncio.sleep(low)

async def get_btns(_loop):
    global cond,rgb
    while True:
        await asyncio.sleep(0.01)
        if btns[0].read() != 0: #red\
            rgb = RED
            all_off()
            cond = False
        await asyncio.sleep(0.01)
        if btns[1].read() != 0: #green
            rgb = GREEN
            all_off()
            cond = False
        await asyncio.sleep(0.01)
        if btns[2].read() != 0: #blue
            rgb = BLUE
            all_off()
            cond = False
        await asyncio.sleep(0.01)
        if btns[3].read() != 0: #OFF
            _loop.stop()

loop = asyncio.new_event_loop()
loop.create_task(flash_red())
loop.create_task(flash_leds())
loop.create_task(get_btns(loop))
loop.run_forever()
loop.close()
all_off()
print("Done.")

Done.
